In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dropout, LSTM, Dense, Add, MultiHeadAttention, Layer, GlobalAveragePooling1D, Concatenate

In [2]:
# Custom CBAM Layer
class CBAM(Layer):
    def __init__(self, reduction_ratio=16, **kwargs):
        super(CBAM, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        channels = input_shape[-1]
        # Channel Attention
        self.shared_dense1 = Dense(channels // self.reduction_ratio, activation='relu')
        self.shared_dense2 = Dense(channels, activation='sigmoid')
        # Spatial Attention
        self.conv = Conv1D(1, kernel_size=7, padding='same', activation='sigmoid')

    def call(self, inputs):
        # Channel Attention
        avg_pool = tf.reduce_mean(inputs, axis=1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=1, keepdims=True)
        avg_out = self.shared_dense2(self.shared_dense1(avg_pool))
        max_out = self.shared_dense2(self.shared_dense1(max_pool))
        channel_attention = avg_out + max_out
        x = inputs * channel_attention

        # Spatial Attention
        avg_spatial = tf.reduce_mean(x, axis=-1, keepdims=True)
        max_spatial = tf.reduce_max(x, axis=-1, keepdims=True)
        spatial_attention = self.conv(tf.concat([avg_spatial, max_spatial], axis=-1))
        x = x * spatial_attention
        return x

# Swell

In [3]:
# Load and preprocess data
df = pd.read_csv("/kaggle/input/stress/dataset/2. final/datasets/hrv/swell/combined/classification/combined-swell-classification-hrv-dataset.csv")
y = df["NasaTLX Label"]
df.drop(columns=["condition", "NasaTLX class", "subject_id", "Condition Label", "NasaTLX Label"], inplace=True)

if df.isnull().any().any():
    raise ValueError("DataFrame contains missing values.")

In [4]:
X= df
y = y

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # (samples, timesteps, 1)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D, Dropout,
                                     LSTM, Concatenate, GlobalAveragePooling1D,
                                     Dense, MultiHeadAttention, UpSampling1D, Cropping1D)
from tensorflow.keras.models import Model


# Utility function to match time steps
def match_time_steps(tensor, ref_tensor):
    diff = tensor.shape[1] - ref_tensor.shape[1]
    if diff > 0:
        return Cropping1D(cropping=(0, diff))(tensor)
    elif diff < 0:
        up_ratio = ref_tensor.shape[1] // tensor.shape[1]
        tensor = UpSampling1D(size=up_ratio)(tensor)
        if tensor.shape[1] != ref_tensor.shape[1]:
            crop =  ref_tensor.shape[1] - tensor.shape[1]
            tensor = Cropping1D(cropping=(0, crop))(tensor)
    return tensor

# Input
input_layer = Input(shape=(X_scaled.shape[1], 1))  # Replace X_scaled with your actual data

# ----- CNN Path -----
cnn1 = Conv1D(32, 3, activation='relu', padding='same')(input_layer)
cnn11 = MaxPooling1D(2)(cnn1)
cnn11 = Dropout(0.25)(cnn11)

cnn2 = Conv1D(64, 3, activation='relu', padding='same')(cnn11)
cnn22 = MaxPooling1D(2)(cnn2)
cnn22_out = Dropout(0.25)(cnn22)

cnn3 = Conv1D(96, 3, activation='relu', padding='same')(cnn22_out)
cnn33 = MaxPooling1D(2)(cnn3)
cnn33_out = Dropout(0.25)(cnn33)

cnn4 = Conv1D(128, 3, activation='relu', padding='same')(cnn33_out)
cnn44 = MaxPooling1D(2)(cnn4)
cnn44_out = Dropout(0.25)(cnn44)

# ----- LSTM Path -----
lstm1 = LSTM(32, return_sequences=True)(input_layer)
lstm11 = MaxPooling1D(2)(lstm1)
lstm11 = Dropout(0.25)(lstm11)

lstm2 = LSTM(64, return_sequences=True)(lstm11)
lstm22 = MaxPooling1D(2)(lstm2)
lstm22_out = Dropout(0.25)(lstm22)

lstm3 = LSTM(96, return_sequences=True)(lstm22_out)
lstm33 = MaxPooling1D(2)(lstm3)
lstm33_out = Dropout(0.25)(lstm33)

lstm4 = LSTM(128, return_sequences=True)(lstm33_out)
lstm44 = MaxPooling1D(2)(lstm4)
lstm44_out = Dropout(0.25)(lstm44)

# ----- Fusion 2: Concatenate CNN2 + LSTM2 -----
#cnn2_down = match_time_steps(cnn2_down, lstm2_out)
combined_2 = Concatenate()([cnn2, lstm2])

# ----- Fusion 3: Multi-head attention + Concatenate -----
combined_3 = Concatenate()([cnn3, lstm3])
combined_3 = MultiHeadAttention(num_heads=4, key_dim=64)(combined_3, combined_3)


# ----- Fusion 4: CBAM + Concatenate -----
combined_4_input = Concatenate()([cnn4, lstm4])
combined_4 = CBAM()(combined_4_input)

# ----- Global Pooling -----
flat_2 = GlobalAveragePooling1D()(combined_2)
flat_3 = GlobalAveragePooling1D()(combined_3)
flat_4 = GlobalAveragePooling1D()(combined_4)

# ----- Final Classification -----
concat = Concatenate()([flat_2, flat_3, flat_4])
output_layer = Dense(3, activation='softmax')(concat)  # Use 3 or your actual num_classes

# ----- Build and Compile -----
model_swell = Model(inputs=input_layer, outputs=output_layer)
model_swell.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_swell.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 64, 32)         │            128 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 64, 32)         │          4,352 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d             │ (None, 32, 32)         │              0 │ conv1d[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_4           │ (None, 32, 32)         │              0 │ lstm[0][0]             │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 32, 32)         │              0 │ max_pooling1d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 32, 32)         │              0 │ max_pooling1d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 32, 64)         │          6,208 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 32, 64)         │         24,832 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_1           │ (None, 16, 64)         │              0 │ conv1d_1[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_5           │ (None, 16, 64)         │              0 │ lstm_1[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 16, 64)         │              0 │ max_pooling1d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 16, 64)         │              0 │ max_pooling1d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 16, 96)         │         18,528 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ (None, 16, 96)         │         61,824 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_2           │ (None, 8, 96)          │              0 │ conv1d_2[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_6           │ (None, 8, 96)          │              0 │ lstm_2[0][0]           │
│ (MaxPooling1D)       

 Total params: 475,842 (1.82 MB)

 Trainable params: 475,842 (1.82 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model_swell.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 165s 18ms/step - accuracy: 0.7623 - loss: 0.5236 - val_accuracy: 0.9960 - val_loss: 0.0160
Epoch 2/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 157s 18ms/step - accuracy: 0.9654 - loss: 0.0982 - val_accuracy: 0.9972 - val_loss: 0.0103
Epoch 3/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 156s 18ms/step - accuracy: 0.9802 - loss: 0.0577 - val_accuracy: 0.9950 - val_loss: 0.0155
Epoch 4/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 157s 18ms/step - accuracy: 0.9849 - loss: 0.0462 - val_accuracy: 0.9997 - val_loss: 0.0015
Epoch 5/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 158s 18ms/step - accuracy: 0.9902 - loss: 0.0302 - val_accuracy: 0.9999 - val_loss: 7.3333e-04
Epoch 6/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 158s 18ms/step - accuracy: 0.9912 - loss: 0.0270 - val_accuracy: 0.9993 - val_loss: 0.0016
Epoch 7/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 157s 18ms/step - accuracy: 0.9928 - loss: 0.0228 - val_accuracy: 0.9996 - val_loss: 0.0012
Epoch 8/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 155s 18ms/step - accura

In [8]:
model_swell.save("Swell_LACCI.h5")

In [10]:
# 7. Evaluate
y_pred = np.argmax(model_swell.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

2448/2448 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     25925
           1       1.00      1.00      1.00     26182
           2       0.99      1.00      0.99     26221

    accuracy                           0.99     78328
   macro avg       0.99      0.99      0.99     78328
weighted avg       0.99      0.99      0.99     78328

Confusion Matrix:
 [[25541     9   375]
 [    0 26178     4]
 [   25    26 26170]]


# ***wesad***

In [11]:
# Load and preprocess data
df = pd.read_csv("/kaggle/input/stress/dataset/2. final/datasets/eda/wesad/combined/classification/wesad-chest-combined-classification-eda.csv")
y = df["SSSQ Label"]
df.drop(columns=["SSSQ class", "condition", "subject id", "condition label", "SSSQ Label"], inplace=True)

if df.isnull().any().any():
    raise ValueError("DataFrame contains missing values.")

In [12]:
X= df
y = y

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D, Dropout,
                                     LSTM, Concatenate, GlobalAveragePooling1D,
                                     Dense, MultiHeadAttention, UpSampling1D, Cropping1D)
from tensorflow.keras.models import Model


# Utility function to match time steps
def match_time_steps(tensor, ref_tensor):
    diff = tensor.shape[1] - ref_tensor.shape[1]
    if diff > 0:
        return Cropping1D(cropping=(0, diff))(tensor)
    elif diff < 0:
        up_ratio = ref_tensor.shape[1] // tensor.shape[1]
        tensor = UpSampling1D(size=up_ratio)(tensor)
        if tensor.shape[1] != ref_tensor.shape[1]:
            crop =  ref_tensor.shape[1] - tensor.shape[1]
            tensor = Cropping1D(cropping=(0, crop))(tensor)
    return tensor

# Input
input_layer = Input(shape=(X_scaled.shape[1], 1))  # Replace X_scaled with your actual data

# ----- CNN Path -----
cnn1 = Conv1D(32, 3, activation='relu', padding='same')(input_layer)
cnn11 = MaxPooling1D(2)(cnn1)
cnn11 = Dropout(0.25)(cnn11)

cnn2 = Conv1D(64, 3, activation='relu', padding='same')(cnn11)
cnn22 = MaxPooling1D(2)(cnn2)
cnn22_out = Dropout(0.25)(cnn22)

cnn3 = Conv1D(96, 3, activation='relu', padding='same')(cnn22_out)
cnn33 = MaxPooling1D(2)(cnn3)
cnn33_out = Dropout(0.25)(cnn33)

cnn4 = Conv1D(128, 3, activation='relu', padding='same')(cnn33_out)
cnn44 = MaxPooling1D(2)(cnn4)
cnn44_out = Dropout(0.25)(cnn44)

# ----- LSTM Path -----
lstm1 = LSTM(32, return_sequences=True)(input_layer)
lstm11 = MaxPooling1D(2)(lstm1)
lstm11 = Dropout(0.25)(lstm11)

lstm2 = LSTM(64, return_sequences=True)(lstm11)
lstm22 = MaxPooling1D(2)(lstm2)
lstm22_out = Dropout(0.25)(lstm22)

lstm3 = LSTM(96, return_sequences=True)(lstm22_out)
lstm33 = MaxPooling1D(2)(lstm3)
lstm33_out = Dropout(0.25)(lstm33)

lstm4 = LSTM(128, return_sequences=True)(lstm33_out)
lstm44 = MaxPooling1D(2)(lstm4)
lstm44_out = Dropout(0.25)(lstm44)

# ----- Fusion 2: Concatenate CNN2 + LSTM2 -----
#cnn2_down = match_time_steps(cnn2_down, lstm2_out)
combined_2 = Concatenate()([cnn2, lstm2])

# ----- Fusion 3: Multi-head attention + Concatenate -----
combined_3 = Concatenate()([cnn3, lstm3])
combined_3 = MultiHeadAttention(num_heads=4, key_dim=64)(combined_3, combined_3)


# ----- Fusion 4: CBAM + Concatenate -----
combined_4_input = Concatenate()([cnn4, lstm4])
combined_4 = CBAM()(combined_4_input)

# ----- Global Pooling -----
flat_2 = GlobalAveragePooling1D()(combined_2)
flat_3 = GlobalAveragePooling1D()(combined_3)
flat_4 = GlobalAveragePooling1D()(combined_4)

# ----- Final Classification -----
concat = Concatenate()([flat_2, flat_3, flat_4])
output_layer = Dense(3, activation='softmax')(concat)  # Use 3 or your actual num_classes

# ----- Build and Compile -----
model_wesad = Model(inputs=input_layer, outputs=output_layer)
model_wesad.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_wesad.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 64, 1)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 64, 32)         │            128 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_4 (LSTM)             │ (None, 64, 32)         │          4,352 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_8           │ (None, 32, 32)         │              0 │ conv1d_5[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_12          │ (None, 32, 32)         │              0 │ lstm_4[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 32, 32)         │              0 │ max_pooling1d_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_13 (Dropout)      │ (None, 32, 32)         │              0 │ max_pooling1d_12[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_6 (Conv1D)         │ (None, 32, 64)         │          6,208 │ dropout_9[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_5 (LSTM)             │ (None, 32, 64)         │         24,832 │ dropout_13[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_9           │ (None, 16, 64)         │              0 │ conv1d_6[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_13          │ (None, 16, 64)         │              0 │ lstm_5[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 16, 64)         │              0 │ max_pooling1d_9[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_14 (Dropout)      │ (None, 16, 64)         │              0 │ max_pooling1d_13[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_7 (Conv1D)         │ (None, 16, 96)         │         18,528 │ dropout_10[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ (None, 16, 96)         │         61,824 │ dropout_14[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_10          │ (None, 8, 96)          │              0 │ conv1d_7[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_14     

 Total params: 475,842 (1.82 MB)

 Trainable params: 475,842 (1.82 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = model_wesad.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 161s 18ms/step - accuracy: 0.7506 - loss: 0.5405 - val_accuracy: 0.9928 - val_loss: 0.0218
Epoch 2/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 156s 18ms/step - accuracy: 0.9669 - loss: 0.0943 - val_accuracy: 0.9977 - val_loss: 0.0075
Epoch 3/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 157s 18ms/step - accuracy: 0.9811 - loss: 0.0544 - val_accuracy: 0.9996 - val_loss: 0.0017
Epoch 4/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 154s 18ms/step - accuracy: 0.9876 - loss: 0.0366 - val_accuracy: 0.9999 - val_loss: 6.6365e-04
Epoch 5/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 153s 17ms/step - accuracy: 0.9902 - loss: 0.0293 - val_accuracy: 0.9999 - val_loss: 6.6669e-04
Epoch 6/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 156s 18ms/step - accuracy: 0.9923 - loss: 0.0240 - val_accuracy: 0.9997 - val_loss: 8.9554e-04
Epoch 7/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 152s 17ms/step - accuracy: 0.9935 - loss: 0.0204 - val_accuracy: 0.9998 - val_loss: 8.0194e-04
Epoch 8/10
8812/8812 ━━━━━━━━━━━━━━━━━━━━ 152s 17ms/s

In [16]:
model_wesad.save("Wesad_LACCI.h5")

In [17]:
# 7. Evaluate
y_pred = np.argmax(model_wesad.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

2448/2448 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     25925
           1       1.00      1.00      1.00     26182
           2       1.00      1.00      1.00     26221

    accuracy                           1.00     78328
   macro avg       1.00      1.00      1.00     78328
weighted avg       1.00      1.00      1.00     78328

Confusion Matrix:
 [[25921     1     3]
 [    0 26179     3]
 [    4     0 26217]]


# ***brainwave*** 

In [18]:
# 1. Load the dataset
df = pd.read_csv('/kaggle/input/brainwave/brainwave_preporcessed.csv')

In [19]:
# 2. Separate features and target
target_col = df.columns[-1]
features_cols = df.columns[:-1]
df[features_cols] = df[features_cols].apply(pd.to_numeric, errors='coerce')
df_clean = df.dropna(subset=features_cols).reset_index(drop=True)

# Display classes
if df_clean[target_col].dtype == 'object':
    print("Classes in the dataset:", df_clean[target_col].unique())
    df_clean[target_col] = df_clean[target_col].astype('category').cat.codes
else:
    print("Classes in the dataset:", np.sort(df_clean[target_col].unique()))

Classes in the dataset: ['NEGATIVE' 'NEUTRAL' 'POSITIVE']


In [20]:
X = df_clean[features_cols]
y = df_clean[target_col]

In [21]:
# 4. Prepare data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D, Dropout,
                                     LSTM, Concatenate, GlobalAveragePooling1D,
                                     Dense, MultiHeadAttention, UpSampling1D, Cropping1D)
from tensorflow.keras.models import Model


# Utility function to match time steps
def match_time_steps(tensor, ref_tensor):
    diff = tensor.shape[1] - ref_tensor.shape[1]
    if diff > 0:
        return Cropping1D(cropping=(0, diff))(tensor)
    elif diff < 0:
        up_ratio = ref_tensor.shape[1] // tensor.shape[1]
        tensor = UpSampling1D(size=up_ratio)(tensor)
        if tensor.shape[1] != ref_tensor.shape[1]:
            crop =  ref_tensor.shape[1] - tensor.shape[1]
            tensor = Cropping1D(cropping=(0, crop))(tensor)
    return tensor

# Input
input_layer = Input(shape=(X_scaled.shape[1], 1))  # Replace X_scaled with your actual data

# ----- CNN Path -----
cnn1 = Conv1D(32, 3, activation='relu', padding='same')(input_layer)
cnn11 = MaxPooling1D(2)(cnn1)
cnn11 = Dropout(0.25)(cnn11)

cnn2 = Conv1D(64, 3, activation='relu', padding='same')(cnn11)
cnn22 = MaxPooling1D(2)(cnn2)
cnn22_out = Dropout(0.25)(cnn22)

cnn3 = Conv1D(96, 3, activation='relu', padding='same')(cnn22_out)
cnn33 = MaxPooling1D(2)(cnn3)
cnn33_out = Dropout(0.25)(cnn33)

cnn4 = Conv1D(128, 3, activation='relu', padding='same')(cnn33_out)
cnn44 = MaxPooling1D(2)(cnn4)
cnn44_out = Dropout(0.25)(cnn44)

# ----- LSTM Path -----
lstm1 = LSTM(32, return_sequences=True)(input_layer)
lstm11 = MaxPooling1D(2)(lstm1)
lstm11 = Dropout(0.25)(lstm11)

lstm2 = LSTM(64, return_sequences=True)(lstm11)
lstm22 = MaxPooling1D(2)(lstm2)
lstm22_out = Dropout(0.25)(lstm22)

lstm3 = LSTM(96, return_sequences=True)(lstm22_out)
lstm33 = MaxPooling1D(2)(lstm3)
lstm33_out = Dropout(0.25)(lstm33)

lstm4 = LSTM(128, return_sequences=True)(lstm33_out)
lstm44 = MaxPooling1D(2)(lstm4)
lstm44_out = Dropout(0.25)(lstm44)

# ----- Fusion 2: Concatenate CNN2 + LSTM2 -----
#cnn2_down = match_time_steps(cnn2_down, lstm2_out)
combined_2 = Concatenate()([cnn2, lstm2])

# ----- Fusion 3: Multi-head attention + Concatenate -----
combined_3 = Concatenate()([cnn3, lstm3])
combined_3 = MultiHeadAttention(num_heads=4, key_dim=64)(combined_3, combined_3)


# ----- Fusion 4: CBAM + Concatenate -----
combined_4_input = Concatenate()([cnn4, lstm4])
combined_4 = CBAM()(combined_4_input)

# ----- Global Pooling -----
flat_2 = GlobalAveragePooling1D()(combined_2)
flat_3 = GlobalAveragePooling1D()(combined_3)
flat_4 = GlobalAveragePooling1D()(combined_4)

# ----- Final Classification -----
concat = Concatenate()([flat_2, flat_3, flat_4])
output_layer = Dense(3, activation='softmax')(concat)  # Use 3 or your actual num_classes

# ----- Build and Compile -----
model_brainwave = Model(inputs=input_layer, outputs=output_layer)
model_brainwave.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_brainwave.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 1516, 1)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_10 (Conv1D)        │ (None, 1516, 32)       │            128 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_8 (LSTM)             │ (None, 1516, 32)       │          4,352 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_16          │ (None, 758, 32)        │              0 │ conv1d_10[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_20          │ (None, 758, 32)        │              0 │ lstm_8[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_18 (Dropout)      │ (None, 758, 32)        │              0 │ max_pooling1d_16[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_22 (Dropout)      │ (None, 758, 32)        │              0 │ max_pooling1d_20[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_11 (Conv1D)        │ (None, 758, 64)        │          6,208 │ dropout_18[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_9 (LSTM)             │ (None, 758, 64)        │         24,832 │ dropout_22[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_17          │ (None, 379, 64)        │              0 │ conv1d_11[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_21          │ (None, 379, 64)        │              0 │ lstm_9[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_19 (Dropout)      │ (None, 379, 64)        │              0 │ max_pooling1d_17[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_23 (Dropout)      │ (None, 379, 64)        │              0 │ max_pooling1d_21[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_12 (Conv1D)        │ (None, 379, 96)        │         18,528 │ dropout_19[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_10 (LSTM)            │ (None, 379, 96)        │         61,824 │ dropout_23[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_18          │ (None, 189, 96)        │              0 │ conv1d_12[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_22     

 Total params: 475,842 (1.82 MB)

 Trainable params: 475,842 (1.82 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model_brainwave.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 14s 148ms/step - accuracy: 0.5918 - loss: 0.7525 - val_accuracy: 0.8655 - val_loss: 0.3267
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 0.8582 - loss: 0.3350 - val_accuracy: 0.8187 - val_loss: 0.3115
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 0.8874 - loss: 0.2492 - val_accuracy: 0.9123 - val_loss: 0.1930
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 0.9079 - loss: 0.2116 - val_accuracy: 0.9181 - val_loss: 0.1953
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 0.9137 - loss: 0.2211 - val_accuracy: 0.9006 - val_loss: 0.2058
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 0.9011 - loss: 0.2287 - val_accuracy: 0.9357 - val_loss: 0.1550
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 0.9021 - loss: 0.2025 - val_accuracy: 0.9415 - val_loss: 0.1364
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 0.9217 - loss: 0.1833 - val_accuracy: 0

In [24]:
model_brainwave.save("Brainwave_LACCI.h5")

In [25]:
# 7. Evaluate
y_pred = np.argmax(model_brainwave.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.92      0.95       143
           1       0.99      1.00      1.00       148
           2       0.92      0.98      0.95       136

    accuracy                           0.97       427
   macro avg       0.97      0.97      0.97       427
weighted avg       0.97      0.97      0.97       427

Confusion Matrix:
 [[132   0  11]
 [  0 148   0]
 [  2   1 133]]


# **SEED**

In [26]:
df = pd.read_csv('/kaggle/input/seed1-preprocessed/seed_1.csv')

In [27]:
# 2. Separate features and target
target_col = df.columns[-1]
features_cols = df.columns[:-1]
df[features_cols] = df[features_cols].apply(pd.to_numeric, errors='coerce')
df_clean = df.dropna(subset=features_cols).reset_index(drop=True)

# Display classes
if df_clean[target_col].dtype == 'object':
    print("Classes in the dataset:", df_clean[target_col].unique())
    df_clean[target_col] = df_clean[target_col].astype('category').cat.codes
else:
    print("Classes in the dataset:", np.sort(df_clean[target_col].unique()))

Classes in the dataset: [0 1 2]


In [28]:
X = df_clean[features_cols]
y = df_clean[target_col]

In [29]:
# 4. Prepare data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [30]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D, Dropout,
                                     LSTM, Concatenate, GlobalAveragePooling1D,
                                     Dense, MultiHeadAttention, UpSampling1D, Cropping1D)
from tensorflow.keras.models import Model


# Utility function to match time steps
def match_time_steps(tensor, ref_tensor):
    diff = tensor.shape[1] - ref_tensor.shape[1]
    if diff > 0:
        return Cropping1D(cropping=(0, diff))(tensor)
    elif diff < 0:
        up_ratio = ref_tensor.shape[1] // tensor.shape[1]
        tensor = UpSampling1D(size=up_ratio)(tensor)
        if tensor.shape[1] != ref_tensor.shape[1]:
            crop =  ref_tensor.shape[1] - tensor.shape[1]
            tensor = Cropping1D(cropping=(0, crop))(tensor)
    return tensor

# Input
input_layer = Input(shape=(X_scaled.shape[1], 1))  # Replace X_scaled with your actual data

# ----- CNN Path -----
cnn1 = Conv1D(32, 3, activation='relu', padding='same')(input_layer)
cnn11 = MaxPooling1D(2)(cnn1)
cnn11 = Dropout(0.25)(cnn11)

cnn2 = Conv1D(64, 3, activation='relu', padding='same')(cnn11)
cnn22 = MaxPooling1D(2)(cnn2)
cnn22_out = Dropout(0.25)(cnn22)

cnn3 = Conv1D(96, 3, activation='relu', padding='same')(cnn22_out)
cnn33 = MaxPooling1D(2)(cnn3)
cnn33_out = Dropout(0.25)(cnn33)

cnn4 = Conv1D(128, 3, activation='relu', padding='same')(cnn33_out)
cnn44 = MaxPooling1D(2)(cnn4)
cnn44_out = Dropout(0.25)(cnn44)

# ----- LSTM Path -----
lstm1 = LSTM(32, return_sequences=True)(input_layer)
lstm11 = MaxPooling1D(2)(lstm1)
lstm11 = Dropout(0.25)(lstm11)

lstm2 = LSTM(64, return_sequences=True)(lstm11)
lstm22 = MaxPooling1D(2)(lstm2)
lstm22_out = Dropout(0.25)(lstm22)

lstm3 = LSTM(96, return_sequences=True)(lstm22_out)
lstm33 = MaxPooling1D(2)(lstm3)
lstm33_out = Dropout(0.25)(lstm33)

lstm4 = LSTM(128, return_sequences=True)(lstm33_out)
lstm44 = MaxPooling1D(2)(lstm4)
lstm44_out = Dropout(0.25)(lstm44)

# ----- Fusion 2: Concatenate CNN2 + LSTM2 -----
#cnn2_down = match_time_steps(cnn2_down, lstm2_out)
combined_2 = Concatenate()([cnn2, lstm2])

# ----- Fusion 3: Multi-head attention + Concatenate -----
combined_3 = Concatenate()([cnn3, lstm3])
combined_3 = MultiHeadAttention(num_heads=4, key_dim=64)(combined_3, combined_3)


# ----- Fusion 4: CBAM + Concatenate -----
combined_4_input = Concatenate()([cnn4, lstm4])
combined_4 = CBAM()(combined_4_input)

# ----- Global Pooling -----
flat_2 = GlobalAveragePooling1D()(combined_2)
flat_3 = GlobalAveragePooling1D()(combined_3)
flat_4 = GlobalAveragePooling1D()(combined_4)

# ----- Final Classification -----
concat = Concatenate()([flat_2, flat_3, flat_4])
output_layer = Dense(3, activation='softmax')(concat)  # Use 3 or your actual num_classes

# ----- Build and Compile -----
model_seed = Model(inputs=input_layer, outputs=output_layer)
model_seed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_seed.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 310, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_15 (Conv1D)        │ (None, 310, 32)        │            128 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_12 (LSTM)            │ (None, 310, 32)        │          4,352 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_24          │ (None, 155, 32)        │              0 │ conv1d_15[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_28          │ (None, 155, 32)        │              0 │ lstm_12[0][0]          │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_27 (Dropout)      │ (None, 155, 32)        │              0 │ max_pooling1d_24[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_31 (Dropout)      │ (None, 155, 32)        │              0 │ max_pooling1d_28[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_16 (Conv1D)        │ (None, 155, 64)        │          6,208 │ dropout_27[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_13 (LSTM)            │ (None, 155, 64)        │         24,832 │ dropout_31[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_25          │ (None, 77, 64)         │              0 │ conv1d_16[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_29          │ (None, 77, 64)         │              0 │ lstm_13[0][0]          │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_28 (Dropout)      │ (None, 77, 64)         │              0 │ max_pooling1d_25[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_32 (Dropout)      │ (None, 77, 64)         │              0 │ max_pooling1d_29[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_17 (Conv1D)        │ (None, 77, 96)         │         18,528 │ dropout_28[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_14 (LSTM)            │ (None, 77, 96)         │         61,824 │ dropout_32[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_26          │ (None, 38, 96)         │              0 │ conv1d_17[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_30     

 Total params: 475,842 (1.82 MB)

 Trainable params: 475,842 (1.82 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# 6. Train the model
history = model_seed.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/10
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 49s 38ms/step - accuracy: 0.4688 - loss: 0.9722 - val_accuracy: 0.7707 - val_loss: 0.5179
Epoch 2/10
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 42s 37ms/step - accuracy: 0.8797 - loss: 0.2968 - val_accuracy: 0.9605 - val_loss: 0.1055
Epoch 3/10
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 42s 37ms/step - accuracy: 0.9560 - loss: 0.1218 - val_accuracy: 0.9890 - val_loss: 0.0333
Epoch 4/10
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 42s 37ms/step - accuracy: 0.9794 - loss: 0.0596 - val_accuracy: 0.9944 - val_loss: 0.0219
Epoch 5/10
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 42s 37ms/step - accuracy: 0.9830 - loss: 0.0497 - val_accuracy: 0.9966 - val_loss: 0.0133
Epoch 6/10
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 42s 37ms/step - accuracy: 0.9910 - loss: 0.0281 - val_accuracy: 0.9956 - val_loss: 0.0205
Epoch 7/10
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 42s 37ms/step - accuracy: 0.9887 - loss: 0.0373 - val_accuracy: 0.9914 - val_loss: 0.0259
Epoch 8/10
1146/1146 ━━━━━━━━━━━━━━━━━━━━ 42s 37ms/step - accuracy: 0.9883 -

In [32]:
model_seed.save("SEED_LACCI.h5")

In [34]:
# 7. Evaluate
y_pred = np.argmax(model_seed.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

319/319 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3301
           1       1.00      1.00      1.00      3372
           2       1.00      1.00      1.00      3509

    accuracy                           1.00     10182
   macro avg       1.00      1.00      1.00     10182
weighted avg       1.00      1.00      1.00     10182

Confusion Matrix:
 [[3301    0    0]
 [   3 3369    0]
 [   0    0 3509]]
